In [ ]:
import pandas as pd
import os 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score

In [ ]:
df = pd.read_pickle('../../data/processed/merged/state0_new_df.pkl')
print(df.shape)

In [ ]:
df = df.drop(columns=['is_ios', 'eventType', 'eventSampleSpeed', 'eventGPSSignalStrength', 'eventSensorDetectionMthd', 'source_file'])
df = df.dropna(subset=['bearingAngleChange'])
corr_matrix = df.corr()